In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd


### Read the input files
#### shape file, hosp dist matrix, hospital coordinates data, block group age, racial and poverty statistics



In [66]:
gdf=gpd.read_file('../Data/Shape_files/ga_bg_county.shp')

bg_dist=pd.read_csv('../Data/BG_Hosp_Dist.csv')

hosp_coord_df = pd.read_csv('../Data/2017ob_hospitals_coords_update.csv', index_col=[0])

tract_age = pd.read_csv('../Data/BG_Agesex/ACSDT5Y2017.B01001-Data.csv')

race_df=pd.read_csv('../Data/Race_stats.csv')
pov_df=pd.read_csv('../Data/Poverty_stats.csv')


<ipython-input-66-e4e2c47231da>:7: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197) have mixed types. Specify dtype option on import or set low_memory=False.
  tract_age = pd.read_csv('../Data/BG_Agesex/ACSDT5Y2017.B01001-Data.csv')


### Convert the data to long format (geo-hosp combinations)

In [37]:
bg_dist_long=pd.melt(bg_dist,id_vars=['County','Geography'],var_name='Hospital',value_name='Distance')


### Merge data sources 

In [39]:
finaldf=gdf.merge(bg_dist_long,how='left',on=['County','Geography'])
finaldf=finaldf.merge(hosp_coord_df[['Hospital','Perinatal Level']],on='Hospital',how='left')
finaldf=finaldf[finaldf['Perinatal Level'].isin(['1','2','3'])]
finaldf=finaldf[finaldf['Perinatal Level']!='Birth Center']

In [41]:
finaldf['Perinatal Level']=finaldf['Perinatal Level'].astype(int)
finaldf=finaldf[['Geography','County','Population','Hospital','Distance','Perinatal Level']].drop_duplicates()


### Retrieve nearest hospital for each block group

In [43]:
idx_min = check.groupby(['County','Geography','Perinatal Level'])['Distance'].idxmin()

# Use the index to retrieve corresponding values from other columns
min_values = check.loc[idx_min]

min_values=min_values.rename(columns={'Hospital':'Org_Min_hospital','Distance':'Org_Min_Distance'})
min_values=min_values.drop(columns=['Population'])


### Merging the min_hosp and min_dist columns back to main structure

In [44]:
finaldf=finaldf.merge(min_values,how='left',on=['County','Geography','Perinatal Level'])


## Race and Poverty data manipulation

In [58]:
race_df=race_df.merge(tract_age[['GEO_ID','NAME']].drop_duplicates(),how='left',left_on='Geographic Area Names',right_on='NAME')
race_df=race_df.merge(gdf[['Geography','Population']],how='left',left_on=['GEO_ID'],right_on='Geography')

race_df=race_df.rename(columns={"Total AmericanIndianORAlaskaNative Female, 18-44":"Native","Total BlackorAfricanAmerican Female, 18-44":"Black","Total HispanicORlatino Female, 18-44":"Hispanic","Total White Female, 18-44":"White",'Total Asian Female, 18-44':'Asian','Total NativeHawaiin&OtherPacificIslander Female, 18-44':'Other'})
race_df=race_df[['Geography','Population','Total Female Age 18-44 years','Native','Black','Hispanic','White','Asian','Other']].drop_duplicates()
race_df['proportion']=(race_df['Population']/race_df['Total Female Age 18-44 years'])

for col in ['Native','Black','Hispanic','White','Asian','Other']:
    race_df[col]=race_df[col]*race_df['proportion']
    race_df[col]=round(race_df[col],0)
    
race_df.drop(columns=['Total Female Age 18-44 years','proportion'],inplace=True)
race_df[race_df['Other']<0]=0


In [59]:
pov_df=pov_df.merge(tract_age[['GEO_ID','NAME']].drop_duplicates(),how='left',left_on='Geographic Area Names',right_on='NAME')
pov_df=pov_df.merge(gdf[['Geography','Population']],how='left',left_on=['GEO_ID'],right_on='Geography')
pov_df=pov_df.drop(columns=['Geography_x','GEO_ID','NAME','Geographic Area Names'])

pov_df.rename(columns={'Income in the past 12 months below poverty level!!Female!!18 to 44 years':'Below_poverty','Income in the past 12 months at or above poverty level!!Female!!18 to 44 years':'Above_poverty','Geography_y':'Geography'},inplace=True)
pov_df['proportion']=(pov_df['Population'])/((pov_df['Below_poverty']+pov_df['Above_poverty']))

for col in ['Below_poverty','Above_poverty']:
    pov_df[col]=pov_df[col]*pov_df['proportion']
    pov_df[col]=round(pov_df[col],0)
pov_df.drop(columns=['proportion'],inplace=True)
